In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from src.paths import LOCAL_PROCESSED_DATA_PATH
from src.processing.dialogre_processing import DialogREDatasetTransformer
from src.processing.dataframe_utils import get_counts_and_percentages
from src.utils import handle_exceptions
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

dt = DialogREDatasetTransformer(LOCAL_PROCESSED_DATA_PATH / 'dialog-re-binary-FIXED')
df = dt.load_data_to_dataframe()
df


,Dialogue,Relations,Origin
0,"[Speaker 1: Hey!, Speaker 2: Hey., Speaker 3: ...","[{'y': 'casting director', 'x': 'Ann', 'rid': ...",dev
1,"[Speaker 1, Speaker 2: Hi, Speaker 3: Hi! Hey ...","[{'y': 'Speaker 2', 'x': 'Speaker 1', 'rid': [...",dev
2,"[Speaker 1, Speaker 2: Hi!, Speaker 3: Hey!, S...","[{'y': 'man', 'x': 'Speaker 4', 'rid': [0], 'r...",dev
3,[Speaker 1: Wow! It looks like we got a lot of...,"[{'y': 'baby', 'x': 'Speaker 2', 'rid': [0], '...",dev
4,"[Speaker 1: Now, Mom, everything's going fine,...","[{'y': '26', 'x': 'Speaker 1', 'rid': [1], 'r'...",dev
...,...,...,...
1783,"[Speaker 1: Nice camoflauge man, for a minute ...","[{'y': 'Speaker 1', 'x': 'Speaker 2', 'rid': [...",train
1784,"[Speaker 1: Well, I'm sure you'll teach her a ...","[{'y': 'Sir', 'x': 'Speaker 1', 'rid': [0], 'r...",train
1785,[Speaker 1: You know what? I can't even worry ...,"[{'y': 'baby', 'x': 'Speaker 1', 'rid': [0], '...",train
1786,"[Speaker 1: And cut. Hey, Butt Guy, what the h...","[{'y': 'Butt Guy', 'x': 'Speaker 2', 'rid': [1...",train


In [79]:
tmp = df.explode('Relations').reset_index()
col = ['index', 'x', 'y', 'r']
tmp = pd.concat([tmp, pd.json_normalize(tmp.Relations)],axis=1)
tmp['r'] = tmp['r'].apply(lambda x: x[0])
tmp = tmp[col]
tmp

,index,x,y,r
0,0,Ann,casting director,with_relation
1,0,Ann,Annie,with_relation
2,0,Estelle,agent,with_relation
3,0,Estelle,Speaker 1,with_relation
4,0,Speaker 2,Katelynn,with_relation
...,...,...,...,...
48702,1787,Speaker 1,Racquela,no_relation_unanswerable
48703,1787,Pheebs,Racquela,no_relation_unanswerable
48704,1787,Speaker 1,Speaker 6,no_relation_unanswerable
48705,1787,Racquela,Speaker 6,with_relation


In [80]:
import pandas as pd

# Assuming tmp is your DataFrame
grouped = tmp.groupby(['index'])

symmetric = True
for name, group in grouped:
    for i in range(len(group)):
        row_i = group.iloc[i]
        x_i, y_i, r_i = row_i['x'], row_i['y'], row_i['r']
        subset = group[(group['x'] == y_i) & (group['y'] == x_i)]
        if not subset.empty:
            r_j = subset['r'].values[0]
            if r_i != r_j:
                symmetric = False
                print(f"Symmetry violation at index {name} between {x_i}, {y_i} and {y_i}, {x_i}. Relation is {r_i} and {r_j} respectively.")
                break
    if not symmetric:
        break

if symmetric:
    print("All relations are symmetric.")


Symmetry violation at index 0 between Speaker 3, man and man, Speaker 3. Relation is no_relation_unanswerable and with_relation respectively.


C:\Users\muril\AppData\Local\Temp\ipykernel_18056\2243424377.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in grouped:


In [81]:
import pandas as pd

# Assuming tmp is your DataFrame
swapped = tmp.copy()
swapped.rename(columns={"x": "y", "y": "x"}, inplace=True)
merged = pd.merge(tmp, swapped, on=['index', 'x', 'y'], how='outer', suffixes=('', '_swapped'))

# Create mask
mask = merged['r'] != merged['r_swapped']

# Display all rows where 'r' values are not symmetric
non_symmetric_rows = merged[mask]

missing_relation_count = non_symmetric_rows.r_swapped.isna().sum()
print('missing_relation_count=',missing_relation_count)

non_symmetric_rows.sort_values(['index', 'x'])


missing_relation_count= 0


,index,x,y,r,r_swapped
9,0,Speaker 3,man,no_relation_unanswerable,with_relation
105,0,man,Speaker 3,with_relation,no_relation_unanswerable
277,2,Geller,Speaker 3,with_relation,no_relation_unanswerable
172,2,Mrs. Geller,Speaker 4,no_relation_unanswerable,with_relation
173,2,Mrs. Geller,Speaker 2,no_relation_unanswerable,with_relation
...,...,...,...,...,...
48990,1785,baby,Speaker 2,with_relation,no_relation_unanswerable
49000,1785,baby,Speaker 1,with_relation,no_relation_unanswerable
48996,1785,little girl,Speaker 1,with_relation,no_relation_unanswerable
49020,1787,Pheebs,Speaker 6,with_relation,no_relation_unanswerable


In [100]:
row_idx = 173
tmp2 = non_symmetric_rows.loc[row_idx]
idx, x, y = tmp2['index'], tmp2['x'], tmp2['y']
mask = (tmp['index'] == 2) & ((tmp['x'] == x) & (tmp['y'] == y) | (tmp['x'] == y) & (tmp['y'] == x))
tmp[mask]

,index,x,y,r
173,2,Mrs. Geller,Speaker 2,no_relation_unanswerable
175,2,Speaker 2,Mrs. Geller,with_relation


In [110]:
tmp['entity_pair'] = tmp.apply(lambda x: sorted([x['x'], x['y']]), axis=1).apply(str)
tmp

,index,x,y,r,entity_pair
0,0,Ann,casting director,with_relation,"['Ann', 'casting director']"
1,0,Ann,Annie,with_relation,"['Ann', 'Annie']"
2,0,Estelle,agent,with_relation,"['Estelle', 'agent']"
3,0,Estelle,Speaker 1,with_relation,"['Estelle', 'Speaker 1']"
4,0,Speaker 2,Katelynn,with_relation,"['Katelynn', 'Speaker 2']"
...,...,...,...,...,...
48702,1787,Speaker 1,Racquela,no_relation_unanswerable,"['Racquela', 'Speaker 1']"
48703,1787,Pheebs,Racquela,no_relation_unanswerable,"['Pheebs', 'Racquela']"
48704,1787,Speaker 1,Speaker 6,no_relation_unanswerable,"['Speaker 1', 'Speaker 6']"
48705,1787,Racquela,Speaker 6,with_relation,"['Racquela', 'Speaker 6']"


In [113]:
tmp.groupby(['index', 'entity_pair']).size().sort_values()

index  entity_pair               
898    ['Speaker 3', 'Speaker 3']    1
170    ['Erica', 'Erica']            1
       ['Jack', 'Jack']              1
602    ['Speaker 3', 'Speaker 3']    1
0      ['Ann', 'Annie']              2
                                    ..
1307   ['Honey', 'Speaker 5']        4
       ['Honey', 'Speaker 4']        4
278    ['Chandler', 'Sweetie']       4
       ['Phoebe', 'Sweetie']         4
1189   ['Knicks', 'Speaker 4']       4
Length: 24280, dtype: int64

In [125]:
d_idx = 1189
x,y = 'Knicks', 'Speaker 4'
raw_dt = DialogREDatasetTransformer(LOCAL_PROCESSED_DATA_PATH / 'dialog-re-with-no-relation-FIXED')
raw_df = raw_dt.load_data_to_dataframe()
raw_tmp = raw_df.explode('Relations').loc[d_idx]
mask = raw_tmp.Relations.apply(lambda r: (r['x'] == x and r['y'] == y) or (r['x'] == y and r['y'] == x) )
raw_tmp[mask].Relations.apply(lambda x: x['r'])

1189    [per:positive_impression]
1189                [no_relation]
1189                [no_relation]
1189           [inverse_relation]
Name: Relations, dtype: object